# 基于物品的协同过滤--ItemCF

In [1]:
users = ["User1","User2","User3","User4","User5"]
items =["Item A","Item B","Item C","Item D","Item E"]

#用户购买记录数据
datasets = [
    [1, 0, 1, 1, 0],
    [1, 0, 0, 1, 1],
    [1, 0, 1, 0, 0],
    [0, 1, 0, 1, 1],
    [1, 1, 1, 0, 1],
]
import pandas as pd
df = pd.DataFrame(datasets, columns=items, index=users)

In [2]:
df

,Item A,Item B,Item C,Item D,Item E
User1,1,0,1,1,0
User2,1,0,0,1,1
User3,1,0,1,0,0
User4,0,1,0,1,1
User5,1,1,1,0,1


In [3]:
from sklearn.metrics import pairwise_distances
item_sim = 1 - pairwise_distances(df.T.values, metric='jaccard')
item_sim = pd.DataFrame(item_sim, columns=items, index=items)

D:\Apps\Anaconda3\lib\site-packages\sklearn\metrics\pairwise.py:1735: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


In [4]:
item_sim

,Item A,Item B,Item C,Item D,Item E
Item A,1.00,0.200000,0.75,0.40,0.400000
Item B,0.20,1.000000,0.25,0.25,0.666667
Item C,0.75,0.250000,1.00,0.20,0.200000
Item D,0.40,0.250000,0.20,1.00,0.500000
Item E,0.40,0.666667,0.20,0.50,1.000000


In [5]:
# 为每个item筛选top2个相似物品
TopN_item = {}
for item in item_sim.index:
    _df = item_sim.loc[item].drop([item])
    _df_sorted = _df.sort_values(ascending=False)
    TopN_item[item] = _df_sorted[:2].index

In [6]:
TopN_item

{'Item A': Index(['Item C', 'Item E'], dtype='object'),
 'Item B': Index(['Item E', 'Item D'], dtype='object'),
 'Item C': Index(['Item A', 'Item B'], dtype='object'),
 'Item D': Index(['Item E', 'Item A'], dtype='object'),
 'Item E': Index(['Item B', 'Item D'], dtype='object')}

In [7]:
import numpy as np
# 为每个用户推荐物品
results = {}
for user in df.index:
    result = set()
    # 用户购买过的物品
    pursed_items = set(df.loc[user].replace(0,np.nan).dropna().index)
    for item in pursed_items:
        result = result.union(set(TopN_item[item]))
    
    # 过滤用户已购买过的物品
    result -= pursed_items
    results[user] = result

In [8]:
results

{'User1': {'Item B', 'Item E'},
 'User2': {'Item B', 'Item C'},
 'User3': {'Item B', 'Item E'},
 'User4': {'Item A'},
 'User5': {'Item D'}}